# Clipper de Áudio (YouTube) — 1 clique

**Como usar:** Preencha os campos no bloco abaixo e clique **▶ Executar**. Só isso.
O próprio bloco prepara o ambiente (instala/checa dependências), baixa o áudio,
recorta e **faz o download automático do MP3**.

Dica: você pode colar uma *Especificação de Clipe* assim: `https://youtu.be/ID?t=123 | 5s`.


In [ ]:
# @title Gerar clipe MP3 (1 clique)
# @markdown Preencha os campos e clique ▶.
from typing import Optional
import sys, subprocess, shutil, re, os, tempfile, uuid

# ====== Formulário ======
especificacao = "" # @param {type:"string"}
youtube_url_or_id = "" # @param {type:"string"}
tempo_inicial = "0"    # @param {type:"string"}
duracao_seg = 5        # @param {type:"number"}
bitrate_kbps = 96      # @param {type:"number"}

def ensure_module(mod_name, pip_name=None):
  try:
    __import__(mod_name)
    return True
  except ImportError:
    pip_name = pip_name or mod_name
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pip_name], check=True)
    __import__(mod_name)
    return True

def ensure_ffmpeg():
  if shutil.which("ffmpeg"):
    return True
  # Tenta instalar no Colab (silencioso)
  subprocess.run(["apt-get", "update", "-y"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
  subprocess.run(["apt-get", "install", "-y", "ffmpeg"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
  return bool(shutil.which("ffmpeg"))

print("Preparando ambiente...")
ensure_module("yt_dlp", "yt-dlp")
ensure_ffmpeg()
from yt_dlp import YoutubeDL
print("Ambiente pronto. Gerando clipe...")

def parse_time_any(s: str) -> float:
  s = (s or "").strip().lower()
  if not s:
    return 0.0
  m = re.search(r"[?&#]t=(\d+)", s)
  if m:
    try: return float(m.group(1))
    except: pass
  if re.search(r"\d+h|\d+m|\d+s", s):
    h = m = sec = 0
    mh = re.search(r"(\d+)h", s)
    mm = re.search(r"(\d+)m", s)
    ms = re.search(r"(\d+)s", s)
    if mh: h = int(mh.group(1))
    if mm: m = int(mm.group(1))
    if ms: sec = int(ms.group(1))
    return float(h*3600 + m*60 + sec)
  if ":" in s:
    parts = [p for p in s.split(":") if p!=""]
    try:
      parts = [float(p) for p in parts]
      if len(parts)==3: return parts[0]*3600 + parts[1]*60 + parts[2]
      if len(parts)==2: return parts[0]*60 + parts[1]
    except: pass
  try:
    return float(s)
  except:
    return 0.0

def parse_duration_any(s: str) -> float:
  s = (s or "").strip().lower()
  if not s: return 0.0
  m = re.search(r"(\d+)\s*s", s)
  if m: return float(m.group(1))
  try: return float(s)
  except: return 0.0

def extract_video_id(url_or_id: str) -> Optional[str]:
  s = (url_or_id or "").strip()
  if not s:
    return None
  m = re.search(r"(?:v=|youtu\.be/|youtube\.com/embed/)([A-Za-z0-9_-]{6,})", s)
  if m: return m.group(1)
  if re.fullmatch(r"[A-Za-z0-9_-]{6,}", s):
    return s
  return None

spec_id = extract_video_id(especificacao)
spec_t  = parse_time_any(especificacao)
spec_d  = parse_duration_any(especificacao)

vid = extract_video_id(youtube_url_or_id) or spec_id
start_sec = parse_time_any(tempo_inicial) or spec_t or 0.0
try:
  dur = float(duracao_seg) if duracao_seg else (spec_d or 5.0)
except:
  dur = spec_d or 5.0

if not vid:
  print("❗ Forneça uma URL/ID válido OU cole uma Especificação de Clipe contendo o link.")
  raise SystemExit

start_sec = max(0.0, start_sec)
dur = max(0.1, dur)

print(f"Vídeo: {vid} | Início: {start_sec:.3f}s | Duração: {dur:.3f}s | Bitrate: {int(bitrate_kbps)}k")

work = tempfile.mkdtemp(prefix="clipper_")
outtmpl = os.path.join(work, "%(id)s.%(ext)s")
ydl_opts = {
    "skip_download": False,
    "format": "bestaudio/best",
    "outtmpl": outtmpl,
    "quiet": True,
    "no_warnings": True,
    "noprogress": True,
}
print("Baixando áudio do YouTube... (aguarde)")
src_path = None
with YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(vid, download=True)
    cand = os.path.join(work, f"{info['id']}")
    for ext in ("m4a","webm","opus","mp4","mkv","mp3"):
        p = cand + "." + ext
        if os.path.exists(p):
            src_path = p
            break
if not src_path:
  print("❗ Não foi possível localizar o arquivo de áudio baixado.")
  raise SystemExit

out_name = f"clip_{vid}_{int(start_sec)}s_{int(dur)}s_{uuid.uuid4().hex[:6]}.mp3"
out_path = os.path.join("/content", out_name)
print("Recortando e gerando MP3...")

cmd = [
    "ffmpeg", "-y",
    "-ss", str(start_sec),
    "-t", str(dur),
    "-i", src_path,
    "-vn",
    "-acodec", "libmp3lame",
    "-b:a", f"{int(bitrate_kbps)}k",
    "-ar", "44100",
    out_path
]
subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, check=True)

print(f"✅ Pronto: {out_path}")
print(f"🔗 YouTube no ponto: https://youtu.be/{vid}?t={int(start_sec)}")

try:
  from google.colab import files
  files.download(out_path)
except Exception:
  print("Se o download não abrir, salve o arquivo pelo painel 'Files' à esquerda.")